<a href="https://colab.research.google.com/github/saitejakarre/Saiteja/blob/main/Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

df = load_dataset("knkarthick/dialogsum")

README.md: 0.00B [00:00, ?B/s]

train.csv:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [ ]:
from transformers import pipeline

text_summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [ ]:
article_1 = df['train'][1]['dialogue']

text_summarizer(article_1, max_length=20, min_length=10, do_sample=False)

[{'summary_text': 'Ricky has received his Polio, Tetanus and Hepatitis B shots.'}]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import TrainingArguments, Trainer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-base")

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [ ]:
#tokenization

def preprocess_function(batch):
    source = batch['dialogue']
    target = batch["summary"]
    source_ids = tokenizer(source, truncation=True, padding="max_length", max_length=128)
    target_ids = tokenizer(target, truncation=True, padding="max_length", max_length=128)

    # Replace pad token id with -100 for labels to ignore padding in loss computation
    labels = target_ids["input_ids"]
    labels = [[(label if label != tokenizer.pad_token_id else -100) for label in labels_example] for labels_example in labels]

    return {
        "input_ids": source_ids["input_ids"],
        "attention_mask": source_ids["attention_mask"],
        "labels": labels
    }

df_source = df.map(preprocess_function, batched=True)

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="/content",  # Replace with your output directory
    per_device_train_batch_size=8,
    num_train_epochs=2,
    report_to='none', # Adjust number of epochs as needed
    remove_unused_columns=False
)

In [ ]:
# Create Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=df_source["train"],
    eval_dataset=df_source["test"]
)

In [ ]:
trainer.train()

Step,Training Loss
500,1.886000
1000,1.740100
1500,1.687500
2000,1.488100
2500,1.442200
3000,1.425800


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3909: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=3116, training_loss=1.6065123365840495, metrics={'train_runtime': 1171.1838, 'train_samples_per_second': 21.278, 'train_steps_per_second': 2.661, 'total_flos': 1899329067417600.0, 'train_loss': 1.6065123365840495, 'epoch': 2.0})

In [ ]:
# Evaluate the model
eval_results = trainer.evaluate()

# Print evaluation results
print(eval_results)

{'eval_loss': 1.6490641832351685, 'eval_runtime': 16.9684, 'eval_samples_per_second': 88.4, 'eval_steps_per_second': 11.079, 'epoch': 2.0}


In [ ]:
# Save the model and tokenizer after training
model.save_pretrained("/content/your_model_directory")
tokenizer.save_pretrained("/content/your_model_directory")

('/content/your_model_directory/tokenizer_config.json',
 '/content/your_model_directory/special_tokens_map.json',
 '/content/your_model_directory/vocab.json',
 '/content/your_model_directory/merges.txt',
 '/content/your_model_directory/added_tokens.json',
 '/content/your_model_directory/tokenizer.json')

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("/content/your_model_directory")
model = AutoModelForSeq2SeqLM.from_pretrained("/content/your_model_directory")

# Function to summarize a blog post
def summarize(blog_post):
    # Tokenize the input blog post
    inputs = tokenizer(blog_post, max_length=1024, truncation=True, return_tensors="pt")

    # Generate the summary
    summary_ids = model.generate(inputs["input_ids"], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)

    # Decode the summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Example blog post
blog_post = """
Lord Venkateshwara, also known as Srinivasa, Balaji, or Venkatachalapati, is a revered form of Lord Vishnu, primarily worshipped at the sacred Tirumala Venkateswara Temple in Tirupati, Andhra Pradesh, India. Considered the “Lord of the Seven Hills,” He is believed to have manifested in the Kali Yuga to protect humanity and grant blessings to His devotees. The deity’s idol is renowned for its unique beauty, adorned with precious jewels and ornaments, and is one of the most visited and venerated forms of Vishnu worldwide.

According to legend, Lord Venkateshwara descended to earth to save mankind from the trials and tribulations of the present age. The most popular story narrates that Vishnu took the form of Srinivasa and came to Tirumala to marry Goddess Padmavathi, the incarnation of Lakshmi. To arrange this divine wedding, He borrowed wealth from Kubera, the god of riches. Even today, devotees believe they are repaying this cosmic debt through offerings at the temple, which explains the tradition of immense donations.

The Tirumala temple is one of the richest and most visited places of worship globally, attracting millions of pilgrims every year. The daily rituals, chants, and the famous “Suprabhatam” prayers bring an atmosphere of deep devotion. The prasadam “Tirupati Laddu” has gained global recognition and holds a Geographical Indication (GI) tag.

Spiritually, Lord Venkateshwara represents hope, compassion, and divine justice. His devotees believe that surrendering at His feet removes sins, fulfills wishes, and grants peace. The sacred mantra “Om Namo Venkatesaya” is chanted with unwavering faith by millions, symbolizing surrender to the Lord’s will.

Beyond being a religious figure, Lord Venkateshwara has influenced culture, music, literature, and traditions for centuries. The annual Brahmotsavam festival draws devotees from across the world, creating a vibrant celebration of faith and unity. His temple stands as a testament to both architectural grandeur and the enduring power of devotion.

In essence, Lord Venkateshwara is not just a deity for His followers; He is a living presence, a protector, and a beacon of hope in challenging times, whose blessings continue to guide devotees toward spiritual fulfillment and worldly prosperity
"""

# Get the summary
summary = summarize(blog_post)
print("Summary:", summary)


Summary: Vishnu is a revered form of Lord Vishnu, primarily worshipped at the sacred Tirumala Venkateswara Temple in Tirupati, Andhra Pradesh, India. He is believed to have manifested in the Kali Yuga to protect humanity and grant blessings to His devotees.
